In [165]:
import pandas as pd
import numpy as np
import math
import regex as re

import altair as alt
from vega_datasets import data

In [166]:
##Reading the data to get tourism indicators 
total_population= pd.read_csv('Dataset_Contract_Sub-Awards.csv')
total_population.head()

,prime_award_unique_key,prime_award_piid,prime_award_parent_piid,prime_award_amount,prime_award_disaster_emergency_fund_codes,prime_award_outlayed_amount_funded_by_COVID-19_supplementals,prime_award_obligated_amount_funded_by_COVID-19_supplementals,prime_award_base_action_date,prime_award_base_action_date_fiscal_year,prime_award_latest_action_date,...,subawardee_highly_compensated_officer_2_name,subawardee_highly_compensated_officer_2_amount,subawardee_highly_compensated_officer_3_name,subawardee_highly_compensated_officer_3_amount,subawardee_highly_compensated_officer_4_name,subawardee_highly_compensated_officer_4_amount,subawardee_highly_compensated_officer_5_name,subawardee_highly_compensated_officer_5_amount,usaspending_permalink,subaward_fsrs_report_last_modified_date
0,CONT_AWD_HR001117C0025_9700_-NONE-_-NONE-,HR001117C0025,NaN,174746702.0,NaN,NaN,NaN,2016-10-31,2017,2021-09-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_HR0...,2021-11-16 13:35:00+00
1,CONT_AWD_HR001117C0025_9700_-NONE-_-NONE-,HR001117C0025,NaN,174746702.0,NaN,NaN,NaN,2016-10-31,2017,2021-09-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_HR0...,2020-11-17 10:56:12+00
2,CONT_AWD_HR001117C0025_9700_-NONE-_-NONE-,HR001117C0025,NaN,174746702.0,NaN,NaN,NaN,2016-10-31,2017,2021-09-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_HR0...,2018-07-09 19:26:13+00
3,CONT_AWD_HR001117C0025_9700_-NONE-_-NONE-,HR001117C0025,NaN,174746702.0,NaN,NaN,NaN,2016-10-31,2017,2021-09-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_HR0...,2018-02-15 14:22:36+00
4,CONT_AWD_HR001117C0025_9700_-NONE-_-NONE-,HR001117C0025,NaN,174746702.0,NaN,NaN,NaN,2016-10-31,2017,2021-09-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_HR0...,2018-03-26 19:23:55+00


In [167]:
#Define a theme for our visualisations
def theme(chart1):
    
    chart1=chart1.configure_axis(
    labelColor="black",
    labelFont="Calibri",
    grid=False
    )

    chart1=chart1.configure_axisLeft(
        titleColor='black',
        ticks=False,
        titlePadding=10,
        labelPadding=10
        ).configure_axisBottom(
          titleColor='black',
          titlePadding=10,
          labelPadding=10,
          ticks=False
        )


    chart1=chart1.configure_title(
        fontSize=16,
        font='Calibri',
        align='center',
        color='black')

    chart1=chart1.configure_view(
        strokeOpacity=0
    )

    return chart1

In [168]:
pop = data.population_engineers_hurricanes()
pop=pop.drop(['population','engineers','hurricanes'],axis=1)
pop.head(10)

,state,id
0,Alabama,1
1,Alaska,2
2,Arizona,4
3,Arkansas,5
4,California,6
5,Colorado,8
6,Connecticut,9
7,Delaware,10
8,District of Columbia,11
9,Florida,12


In [169]:
total_population = pd.merge(total_population,pop,left_on='subawardee_state_name',right_on = "state",how='outer',indicator=True)

In [170]:
total_population2 = (total_population.groupby(['subawardee_state_name']).size()).to_frame()
total_population2=total_population2.reset_index()
total_population2.columns = ['state','num_contracts']

In [171]:
total_population2

,state,num_contracts
0,Alabama,14
1,Arizona,73
2,California,68
3,Colorado,8
4,Connecticut,7
5,Florida,5
6,Illinois,11
7,Iowa,3
8,Maryland,26
9,Massachusetts,11


In [172]:
final_df = pd.merge(pop,total_population2,on='state',how='outer')

In [173]:
final_df.head()

,state,id,num_contracts
0,Alabama,1,14.0
1,Alaska,2,NaN
2,Arizona,4,73.0
3,Arkansas,5,NaN
4,California,6,68.0


In [174]:
final_df['num_contracts'] = final_df['num_contracts'].replace(np.nan, 0)

In [175]:
type(total_population["subawardee_business_types"][0]) == str

True

In [176]:
def change_list(list1):
    if type(list1) == str:
        list2 = list1.split(",")
        return list2
    return np.nan
    

total_population["subawardee_business_types_list"] = total_population["subawardee_business_types"].apply(lambda x: change_list(x))

In [177]:
total_population3 = total_population.explode("subawardee_business_types_list")

In [178]:
total_population3.subawardee_business_types_list

0      For-Profit Organization
0        Manufacturer of Goods
1      For-Profit Organization
1        Manufacturer of Goods
2      For-Profit Organization
                ...           
371                        NaN
372                        NaN
373                        NaN
374                        NaN
375                        NaN
Name: subawardee_business_types_list, Length: 898, dtype: object

In [179]:
total_population3['subawardee_business_types_list'] = total_population3["subawardee_business_types_list"].str.replace(".","")
total_population3['subawardee_business_types_list'] = total_population3["subawardee_business_types_list"].str.replace(" ","")
total_population4 = (total_population3.groupby(['state','subawardee_business_types_list']).size()).to_frame()
total_population4=total_population4.reset_index()
total_population4.columns = ['state','description','num_contracts']
total_population4.head()

C:\Users\sonal\AppData\Local\Temp/ipykernel_2080/2269059591.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  total_population3['subawardee_business_types_list'] = total_population3["subawardee_business_types_list"].str.replace(".","")


,state,description,num_contracts
0,Alabama,For-ProfitOrganization,14
1,Alabama,ManufacturerofGoods,9
2,Alabama,SCorporation,1
3,Arizona,Contracts,1
4,Arizona,ContractsandGrants,2


In [180]:
total_population4[total_population4["description"] == "For-ProfitOrganization"]

,state,description,num_contracts
0,Alabama,For-ProfitOrganization,14
5,Arizona,For-ProfitOrganization,73
19,California,For-ProfitOrganization,68
26,Colorado,For-ProfitOrganization,8
30,Connecticut,For-ProfitOrganization,7
32,Florida,For-ProfitOrganization,5
35,Illinois,For-ProfitOrganization,11
38,Iowa,For-ProfitOrganization,3
41,Maryland,For-ProfitOrganization,26
45,Massachusetts,For-ProfitOrganization,11


In [181]:
final_df2 = pd.merge(pop,total_population4,on='state',how='outer')

In [182]:
final_df2

,state,id,description,num_contracts
0,Alabama,1,For-ProfitOrganization,14.0
1,Alabama,1,ManufacturerofGoods,9.0
2,Alabama,1,SCorporation,1.0
3,Alaska,2,NaN,NaN
4,Arizona,4,Contracts,1.0
...,...,...,...,...
131,West Virginia,54,ManufacturerofGoods,2.0
132,Wisconsin,55,For-ProfitOrganization,10.0
133,Wisconsin,55,ManufacturerofGoods,10.0
134,Wyoming,56,NaN,NaN


In [191]:
selection = alt.selection_multi(fields=['state'])
#init=[{"state":"California"}])

click = alt.selection_multi()
states = alt.topo_feature(data.us_10m.url, 'states')

map1=alt.Chart(states).mark_geoshape().encode(
   color=alt.condition(selection, 'num_contracts:Q', alt.value('lightyellow'),scale=alt.Scale(range=['#b22222','#fff347']),
            legend=alt.Legend(title="Number of Sub-Contracts",titleColor='#4D4C4C',labelColor='#4D4C4C')),

    tooltip=['state:N','num_contracts:Q']
 ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(final_df, 'id', list(final_df.columns))
 ).project(
    type='albersUsa'
 ).add_selection(
     selection)




#map1.save("plots/debt_ratio_state.html")
map1

alt.Chart(...)

In [192]:
bars = alt.Chart(final_df2).mark_bar().encode(
    y=alt.Y('description',title='Sub-Contract Business Type', sort='-x'),
    x=alt.X('num_contracts' ,title='Number of Sub-Contracts'),
    color= alt.Color('state:N',scale=alt.Scale(scheme="Oranges"),legend=None),
    #color = alt.condition(selection,'country:N', alt.value('grey'),scale=alt.Scale(scheme="redyellowblue")),
    tooltip=[alt.Tooltip("num_contracts:Q", title="Number of Sub-Contracts"),
            alt.Tooltip("state:N", title="State")]    
).add_selection(
     selection
 ).transform_filter(
   selection
).properties(title='Sub-Contract Business Type')

In [185]:
bars

alt.Chart(...)

In [193]:
total_participation_viz=alt.vconcat(map1, bars).properties(
    title={
      "text": ["Georgraphic Distribution of Sub Contracts"],
       "subtitle": ["Hover over the States to look at the total contracts","Click on any State to look at the number of sub-contracts by Business Type"],
        "subtitleColor": "#4D4C4C",
      "subtitleFontSize":10,
      #'subtitleFontStyle':'italic',
      "color": "black",
    }
).configure_title(
        fontSize=16,
        font='Calibri',
        align='center',
        color='black').configure(background='#F6F5F5')

In [194]:
total_participation_viz = theme(total_participation_viz)

In [195]:
total_participation_viz

alt.VConcatChart(...)

In [189]:
total_participation_viz.save("map_subcontracts.html")

In [209]:
#total_population3.to_csv("../data/munged_avi.csv" , index=False)